In [1]:
from proxy_chrome_browser import get_chromedriver, build_proxy_extension
import moviepy.editor as mpy
import glob
import os
from time import sleep
from tqdm import tqdm

In [2]:
br = get_chromedriver(headless=True,path='/home/miko/Downloads/chromedriver-v-110')

In [4]:
def find_elm(tag,driver=br,_all=False,_by="css"):
    from selenium.webdriver.common.by import By
    if _by=="css":
        _by=By.CSS_SELECTOR
    elif _by=="link_text":
        _by=By.PARTIAL_LINK_TEXT
    elif _by=="link_text_exact":
        _by=By.LINK_TEXT
        
    try:
        if _all:
            out = driver.find_elements(by=_by,value=tag)
        else:
            out = driver.find_element(by=_by,value=tag)
    except:
        #print('None')
        out = None
    return out

In [5]:
def save_screenshot(elm,filename,scrollY=None):
    assert filename[-3:] == "png", "filename should end with png"
    scroll_into_view(elm,scrollY)
    with open(filename,"wb") as f:
        f.write(elm.screenshot_as_png)

In [6]:
def scroll_into_view(elm,scrollY=None):
     #source: -https://stackoverflow.com/questions/25733877/selenium-how-do-you-check-scroll-position
     #loc = elm.location
     #current_view_script = """ var doc = document.documentElement;
     #var left = (window.pageXOffset || doc.scrollLeft) - (doc.clientLeft || 0);
     #var top = (window.pageYOffset || doc.scrollTop)  - (doc.clientTop || 0);
        
     #taking screenshot will scroll into view. IF this stops working can use javascript above
     elm.screenshot_as_png
     sleep(.2)
     # scroll a bit to add margin (prevent floating login bar from obscuring elm);
     if scrollY:
         br.execute_script(f"window.scrollBy(0,{scrollY})", "")
         sleep(.5)


In [7]:
#set to dark theme
br.get("https://www.reddit.com")

In [8]:
icon = find_elm("button[id='USER_DROPDOWN_ID']")
icon.click()
sleep(.5)
dark_btn = find_elm("button[data-redditstyle='true']")
dark_btn.click()
sleep(.5)
# if for some reason click isnt working. set it manually
#input("Confirm darkmode has been set (press enter)")

ElementClickInterceptedException: Message: element click intercepted: Element <button aria-expanded="false" aria-haspopup="true" id="USER_DROPDOWN_ID" class="_10K5i7NW6qcm-UoCtpB3aK _1pA8z73SZ1olP5KMKFN4_Z _18X7KoiaLuKbuLqg4zE8BH _22SL37yETIW414yUiZj27w">...</button> is not clickable at point (1821, 23). Other element would receive the click: <div class="_1DK52RbaamLOWw5UPaht_S _3Ig_EsWWVLquWs2yBBQjec _1acwN_tUhJ8w-n7oCp-Aw3">...</div>
  (Session info: chrome=113.0.5672.92)
Stacktrace:
#0 0x55701f7a9133 <unknown>
#1 0x55701f4dd966 <unknown>
#2 0x55701f5203f2 <unknown>
#3 0x55701f51e5ac <unknown>
#4 0x55701f51c3be <unknown>
#5 0x55701f51b696 <unknown>
#6 0x55701f5103ea <unknown>
#7 0x55701f539bd2 <unknown>
#8 0x55701f50fd13 <unknown>
#9 0x55701f539d9e <unknown>
#10 0x55701f5521c7 <unknown>
#11 0x55701f5399a3 <unknown>
#12 0x55701f50e46a <unknown>
#13 0x55701f50f55e <unknown>
#14 0x55701f768cae <unknown>
#15 0x55701f76c8fe <unknown>
#16 0x55701f775f20 <unknown>
#17 0x55701f76d923 <unknow

In [9]:
#post_url = "https://www.reddit.com/r/AskReddit/comments/ypzsqh/men_of_reddit_what_are_some_less_obvious_red/"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ycb6hp/whats_a_harsh_reality_that_everybody_needs_to_hear/"
#post_url = "https://www.reddit.com/r/AskReddit/comments/y5bcsl/nonamericans_what_do_you_think_every_american/"
#post_url ="https://www.reddit.com/r/AskReddit/comments/ydz1j5/what_is_25_years_too_old_for/"
#post_url = "https://www.reddit.com/r/AskReddit/comments/y5wxnn/what_is_the_stupidest_thing_a_large_amount_of/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ynpnd5/serious_what_stole_your_joy_why_do_you_not_have/?sort=confid?ence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/yabrxc/whats_a_subtle_sign_of_low_intelligence/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/yuibit/what_job_contributes_nothing_to_society/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/yxxs30/who_had_the_biggest_fall_from_grace_in_history/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ym1lxv/reddit_whats_your_most_im_with_the_boomers_on/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ylvk5f/what_was_great_advice_20_years_ago_but_definitely/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ypu67r/whats_a_hill_youre_willing_to_die_on_every_time/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ywemuo/americans_of_reddit_what_is_the_worst_big_city_in/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ym1lxv/reddit_whats_your_most_im_with_the_boomers_on/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z40utz/what_did_the_pandemic_ruin_more_than_we_realise/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z5neg5/what_tv_show_never_had_a_decline_in_quality/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z1l0m4/what_was_the_saddest_fictional_character_death/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z6l6xn/how_much_do_someones_political_views_matter_when/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ykxnob/what_started_as_a_joke_but_people_take_it_way_to/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/yvxhc3/whats_the_one_thing_you_would_never_buy_second/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/xfcbr4/what_villain_was_terrifying_because_they_were/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/vfq7z9/whats_a_modern_day_scam_thats_become_normalized/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/vhki66/what_improved_your_life_so_much_you_wished_you/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/vjpkye/the_supreme_court_has_overturned_roe_v_wade_how/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z3uzhi/what_ruined_your_thanksgiving_this_year/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z4wbcb/whats_the_best_cartoon_show_of_all_time/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z8ecob/who_is_the_female_vocal_equivalent_of_morgan/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z4pq24/what_celebrity_death_was_the_most_unexpected/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/z78r84/if_you_invented_a_car_that_ran_on_stupidity_where/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zfew2z/whats_a_hobby_someone_can_have_that_is_an/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zhp9r1/whats_a_womens_thing_men_should_absolutely_start/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zeuksw/what_should_you_never_do_in_the_united_states/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ze76ah/whats_the_worst_mistake_youve_seen_someone_do_in/?sort=confidence"
#post_url =  "https://www.reddit.com/r/AskReddit/comments/ziqmsh/what_dark_secret_are_you_hiding_from_everyone/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ziqmsh/what_screams_insecurity_but_most_people_dont/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zk8wn9/not_using_110_how_attractive_are_you/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zlq3aa/those_who_havent_caught_covid_yet_how_have_you/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zo6zuv/if_you_had_the_ability_to_bring_back_a_canceled/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zm4u9z/us_wants_to_ban_tiktok_what_are_your_thoughts/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zd6pxv/whats_something_we_all_just_pretend_no_one_does/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zuzhgo/what_screams_im_a_bad_parent/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zsqivw/what_is_an_american_thing_but_americans_think/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zquyhv/what_was_ruined_by_rich_people/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zpo3so/what_is_so_ridiculously_overpriced_yet_you_still/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zoynqc/what_was_ruined_because_too_many_people_did_it/?sort=confidence"

#post_url = "https://www.reddit.com/r/AskReddit/comments/zy5kmq/what_fact_are_you_just_tired_of_explaining_to/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zzui88/what_do_we_need_to_stop_teaching_the_children/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/103a4x5/whats_a_sound_you_heard_when_you_were_young_that/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/1025m0g/people_of_reddit_what_do_you_mis_about_the_covid/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10034fk/what_company_would_you_like_to_go_bankrupt/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zwv8pq/youve_just_won_250_million_and_its_been_deposited/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zuckzx/what_does_europe_have_that_north_america_does_not/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zsl0mj/what_made_you_not_want_to_have_kids/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ztsagt/what_is_something_acceptable_in_women_but_frowned/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zi7edc/whats_one_of_lifes_biggest_traps_that_people_fall/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/zppkqv/what_joke_is_starting_to_get_old_now/?sort=confidence"

#post_url = "https://www.reddit.com/r/AskReddit/comments/zshbwg/if_you_had_the_power_to_shut_down_a_company_for/?sort=confidence"

#2023-01-23
#post_url = "https://www.reddit.com/r/AskReddit/comments/10dm92h/whats_a_company_secret_you_can_share_now_that_you/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10axb50/what_quietly_went_away_without_anyone_noticing/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10h0r47/what_was_once_highly_respected_that_is_now_a/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10g9cjg/whats_something_you_learned_embarrassingly_late/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/103o62o/you_just_won_a_billion_dollars_what_are_you_still/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/1061sre/people_who_dont_exercise_or_workout_why_dont_you/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10ej6ap/people_of_reddit_who_are_anti_weed_why/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/106uxfp/what_is_a_part_of_history_that_we_consider_to_be/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/10anf4u/what_gets_more_hate_than_it_should/?sort=confidence"
#post_url = "https://www.reddit.com/r/AskReddit/comments/105jfmx/whats_a_serious_event_in_the_world_that_few/?sort=confidence"

# post_urls = [
#    "https://www.reddit.com/r/AskReddit/comments/10qif8x/have_you_ever_listened_to_a_person_talk_for_less/",
#    "https://www.reddit.com/r/AskReddit/comments/10njbup/what_was_your_it_cant_be_that_easy_it_was_that/",
#     "https://www.reddit.com/r/AskReddit/comments/10iqvtp/whats_the_worst_part_of_having_a_child/",
#     "https://www.reddit.com/r/AskReddit/comments/10ypyd0/whos_the_worst_main_character_were_supposed_to/",
#     "https://www.reddit.com/r/AskReddit/comments/10p4k1c/what_screams_this_person_peaked_in_high_school_to/",
#     "https://www.reddit.com/r/AskReddit/comments/10je7cc/what_widelyaccepted_reddit_tropes_are_just_not/",
#     "https://www.reddit.com/r/AskReddit/comments/10kv4yc/what_hobby_is_an_immediate_red_flag/",
#     "https://www.reddit.com/r/AskReddit/comments/10wtgtt/what_industry_do_you_hope_wont_exist_in_10_years/",
#     "https://www.reddit.com/r/AskReddit/comments/10udqhv/what_film_made_you_say_holy_shit_there_is_still/",
#     "https://www.reddit.com/r/AskReddit/comments/10rsog2/what_ingredient_ruins_a_sandwich_for_you/",
#     "https://www.reddit.com/r/AskReddit/comments/10lr2nb/what_could_be_done_to_prevent_mass_shootings/",
#     "https://www.reddit.com/r/AskReddit/comments/10bp478/whats_the_most_boring_sport_to_watch/",
#     "https://www.reddit.com/r/AskReddit/comments/10tu4jz/what_is_a_dead_giveaway_that_youre_being_cheated/"
# ]


# post_urls = [
#   "https://www.reddit.com/r/AskReddit/comments/1128hq1/what_actor_or_actress_stupidly_left_a_movie/"
#   ,"https://www.reddit.com/r/AskReddit/comments/116aaro/whats_a_story_where_the_bad_guys_are_actually/"
#   ,"https://www.reddit.com/r/AskReddit/comments/113uw7u/what_job_position_is_100_overvalued_and_overpaid/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11ck2oh/whats_the_dumbest_myth_people_today_believe/"
#   ,"https://www.reddit.com/r/AskReddit/comments/110kh3j/what_industry_do_you_consider_to_be_legal/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11h681p/whats_an_unwritten_rule_about_the_road_that_new/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11gal39/americans_who_are_against_the_biden_student_loan/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11btq7y/what_is_the_most_bullshit_profession_that/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11artla/what_do_you_think_of_the_parents_that_kick_their/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11a6s9b/which_hobbies_that_people_do_screams_rich_people/"
#   ,"https://www.reddit.com/r/AskReddit/comments/11hy6y8/what_is_your_first_thought_about_someone_when/"
#   ,"https://www.reddit.com/r/AskReddit/comments/10zt10t/pro_gun_supporters_why_do_you_support_gun_rights/"
    
# ]

# post_urls = [
# "https://www.reddit.com/r/AskReddit/comments/11zmpjf/if_you_could_place_any_object_on_the_surface_of/",
# "https://www.reddit.com/r/AskReddit/comments/11jhj8o/how_old_are_you_and_whats_your_biggest_problem/",
# "https://www.reddit.com/r/AskReddit/comments/11wo9g4/if_trump_is_arrested_how_do_you_think_his/",
# "https://www.reddit.com/r/AskReddit/comments/11onl9l/which_profession_attracts_the_worst_kinds_of/",
# "https://www.reddit.com/r/AskReddit/comments/11m0jsp/what_instantly_ruins_a_burger_for_you/",
# "https://www.reddit.com/r/AskReddit/comments/120j3i5/which_cancelled_tv_show_deserved_another_season/",
# "https://www.reddit.com/r/AskReddit/comments/120j3i5/which_cancelled_tv_show_deserved_another_season/",
# "https://www.reddit.com/r/AskReddit/comments/11ujqfu/americans_what_is_something_us_europeans_arent/",
# "https://www.reddit.com/r/AskReddit/comments/120ucjy/you_wake_up_its_january_1st_1999_what_do_you_do/",
    
# ]

# post_urls = [
# "https://www.reddit.com/r/AskReddit/comments/124wb2n/what_instantly_kills_the_vibe_at_a_party/",
# "https://www.reddit.com/r/AskReddit/comments/12acoep/what_is_a_tell_tale_sign_that_someone_is_a/",
# "https://www.reddit.com/r/AskReddit/comments/129k7ok/whats_the_first_sign_that_a_movie_is_going_to_be/",
# "https://www.reddit.com/r/AskReddit/comments/125r99g/what_scientific_fact_scares_the_absolute_shit_out/",
# "https://www.reddit.com/r/AskReddit/comments/124oh82/what_movie_did_you_expect_to_be_bad_but_was/",
# "https://www.reddit.com/r/AskReddit/comments/11qbywk/whats_the_hardest_thing_for_you_to_comprehend/",
# "https://www.reddit.com/r/AskReddit/comments/120j3i5/which_cancelled_tv_show_deserved_another_season/",
# "https://www.reddit.com/r/AskReddit/comments/11vmkfc/americans_what_do_eurpoeans_have_everyday_that/",
# "https://www.reddit.com/r/AskReddit/comments/121joyd/people_who_dont_reply_until_days_later_why/",
    
# ]

# post_urls = [
# "https://www.reddit.com/r/AskReddit/comments/12br1id/how_is_everyone_feeling_about_donald_trump/",
# "https://www.reddit.com/r/AskReddit/comments/12j384c/whats_a_podcast_actually_worth_listening_to/",
# "https://www.reddit.com/r/AskReddit/comments/12cmf8s/what_was_discontinued_but_you_miss_like_hell_and/",
# "https://www.reddit.com/r/AskReddit/comments/11vd3w7/what_famous_person_didnt_deserve_all_the_hate/",
# "https://www.reddit.com/r/AskReddit/comments/122a253/how_do_you_tell_your_gf_she_has_a_mustache/",
# "https://www.reddit.com/r/AskReddit/comments/12emr13/what_show_stayed_good_from_start_to_finish/",
# "https://www.reddit.com/r/AskReddit/comments/12jnimp/what_made_you_realize_youre_old/",
# "https://www.reddit.com/r/AskReddit/comments/12oskxc/what_was_the_weirdest_part_of_the_pandemic/",
# "https://www.reddit.com/r/AskReddit/comments/12bv3w8/anime_is_insanely_popular_in_america_whats_an/",
# "https://www.reddit.com/r/AskReddit/comments/11zmpjf/if_you_could_place_any_object_on_the_surface_of/"
    
# ]

##2023-05-05
# post_urls = [
#     "https://www.reddit.com/r/AskReddit/comments/136jqkc/bernie_sanders_says_us_should_confiscate_100_of/",
#     "https://www.reddit.com/r/AskReddit/comments/134qerl/richard_feynman_said_never_confuse_education_with/",
#     "https://www.reddit.com/r/AskReddit/comments/12rt24x/redditors_who_have_actually_won_a_lifetime_supply/",
#     "https://www.reddit.com/r/AskReddit/comments/12rk46t/there_is_a_greek_proverb_a_society_grows_great/",
#     "https://www.reddit.com/r/AskReddit/comments/12hmbug/what_are_some_really_dangerous_things_most_people/",
#     "https://www.reddit.com/r/AskReddit/comments/131qb3k/whats_the_weirdest_thing_youve_been_told_not_to/",
#     "https://www.reddit.com/r/AskReddit/comments/12jt9df/whats_the_hardest_would_u_rather_question_youve/",
#     "https://www.reddit.com/r/AskReddit/comments/12xrtrd/surgeonsdoctors_of_redditwhats_the_dumbest_thing/",
#     "https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/",
#     "https://www.reddit.com/r/AskReddit/comments/132dbsu/whats_something_that_changeddisappeared_because/",
#     "https://www.reddit.com/r/AskReddit/comments/135h5ah/whats_the_dumbest_thing_youve_done_in_your_20s/",
#     "https://www.reddit.com/r/AskReddit/comments/12j4cxi/what_is_your_one_piece_of_street_advice/", 
# ]

# #2023-05-22
# post_urls = [
#     "https://www.reddit.com/r/AskReddit/comments/13ec6op/has_anyone_ever_been_to_a_wedding_where_someone/",
#     "https://www.reddit.com/r/AskReddit/comments/13i9ark/what_television_series_had_the_biggest_bullshit/",
#     "https://www.reddit.com/r/AskReddit/comments/13lvmly/what_was_normal_2030_years_ago_but_is_considered/",
#     "https://www.reddit.com/r/AskReddit/comments/13kuo93/to_you_redditors_aged_50_whats_something_you/",
#     "https://www.reddit.com/r/AskReddit/comments/13angrv/whats_something_popular_that_you_refuse_to_get/",
#     "https://www.reddit.com/r/AskReddit/comments/13nps5d/people_who_wake_up_at_5am_daily_why/",
#     "https://www.reddit.com/r/AskReddit/comments/13bt91m/what_is_just_not_the_flex_that_people_think_it_is/",
#     "https://www.reddit.com/r/AskReddit/comments/13hyxco/people_who_get_in_their_cars_but_then_take_ages/"
# ]

#2023-06-14
post_urls = [
    "https://www.reddit.com/r/AskReddit/comments/13x9sy7/now_that_reddit_are_killing_3rd_party_apps_on/",
    "https://www.reddit.com/r/AskReddit/comments/13vwcge/whats_the_most_disturbing_secret_youve_discovered/",
    "https://www.reddit.com/r/AskReddit/comments/140xj5s/adults_who_carry_around_a_backpack_whatcha_got_in/",
    "https://www.reddit.com/r/AskReddit/comments/147rws8/wedding_photographers_of_reddit_what_was_your/",
   "https://www.reddit.com/r/AskReddit/comments/141dfp5/what_is_a_weird_flex_you_are_proud_of/",
    "https://www.reddit.com/r/AskReddit/comments/13yu76f/what_are_the_cons_of_not_having_kids/",
    "https://www.reddit.com/r/AskReddit/comments/13vo174/what_is_something_that_old_people_love_that_you/",
    "https://www.reddit.com/r/AskReddit/comments/13s9y0x/would_you_feel_safer_in_a_gunfree_state_why_or/",
    "https://www.reddit.com/r/AskReddit/comments/13rcodf/what_is_your_favourite_insult_that_doesnt_sound/",
   "https://www.reddit.com/r/AskReddit/comments/13qk9vl/what_movie_blew_your_mind_when_you_first_saw_it/",
    "https://www.reddit.com/r/AskReddit/comments/141tabx/what_type_of_human_behavior_will_you_never/",
    "https://www.reddit.com/r/AskReddit/comments/141tdwu/what_urban_legend_needs_to_die/",
    "https://www.reddit.com/r/AskReddit/comments/13u2sgh/what_is_something_that_young_people_love_that_you/",
   "https://www.reddit.com/r/AskReddit/comments/145zqnb/whats_an_important_lesson_you_learnt_the_hard_way/",
   "https://www.reddit.com/r/AskReddit/comments/13re7ii/what_youtube_channel_is_a_goldmine_of/",
    "https://www.reddit.com/r/AskReddit/comments/142kyjh/what_is_your_opinion_on_the_reddit_blackout_and/",
    
   
]





In [10]:
skip=[
]
for post_url in post_urls:
    if post_url in skip:
        continue
        
    post_url+="?sort=confidence"
    br.get(post_url)

    #br.switch_to.active_element.text

    sleep(10)

    # expand replies
    num_replies_threshold = 90
    get_replies = True
    num_total_loops = 5
    current_loop_num = 0
    clicked_links = []
    while get_replies:
        more_replies_divs = find_elm("div[id^='moreComments']",_all=True)
        expandable_divs = []
        for div in more_replies_divs:
            more_replies_link = find_elm("p",driver=div)
            #p = br.execute_script("return arguments[0].parentNode;",more_replies_link)
            if not more_replies_link:
                continue
            if more_replies_link.text == '' or 'loading..' in more_replies_link.text:
                continue
            num_replies = int(more_replies_link.text.split()[0].replace(",",""))
            if num_replies >= num_replies_threshold:
                if more_replies_link not in clicked_links:
                    expandable_divs.append(more_replies_link)
                else:
                    continue
        for div in expandable_divs:
            #link_elm = find_elm("div[data-testid^='search-scope-pill-text']")
            try:
                scroll_into_view(more_replies_link)
                div.click()
                sleep(1)
            except:
                continue  
        if len(expandable_divs) == 0:
            get_replies = False
        if current_loop_num >= num_total_loops:
            get_replies = False
        else:
            current_loop_num += 1
        print("getting more replies")
        #todo: find a better stopping condition

    # get comments
    comments = find_elm("div[class*='Comment']",_all=True)

    #prepend original post to comments
    post = [find_elm("div[data-test-id='post-content']"),]
    post.extend(comments)
    comments = post

    len(comments)

    submission_title = post_url.split("/")[-2]

    #asset_dir="assets/demo_png_2_vid"
    asset_dir="/mnt/n/data/reddit-yt-tts/queued"
    completed_dir="/mnt/n/data/reddit-yt-tts/completed"
    #thread_ids = []

    submission_title

    def save_comment_text(filename,comment,selector='p'):
        comment_text = " ".join([x.text for x in find_elm(selector,driver=comment,_all=True)])
        with open(filename,"w") as f:
            f.write(comment_text)

    #save threaded comment screenshot as png to directory
    vote_count_threshold = 450
    thread_count = 0
    parent_comment_stack = [dict(comment=comments[0],saved=True)]
    prev_comment_lvl = 1
    prev_comment = comments[0]
    prev_comment_id = -1
    i = 0
    for comment in tqdm(comments):
        # filter deleted comments
        if "Comment deleted by user" in comment.text:
            continue

        # some comments arnt expanded, skipping
        if len(comment.text.split("\n")) <= 4:
               continue

        #filter highly voted comments
        vote_div = find_elm("div[id^='vote-arrows']",driver=comment)
        if not vote_div or vote_div.text == '' or vote_div.text == "Vote":
            continue

        multiply_1k = 'k' in vote_div.text
        if multiply_1k:
            vote_count = float(vote_div.text.replace("k","")) * 1000
        else:
            vote_count = int(vote_div.text)

        if i == 0:
            comment_lvl = 1
        else:
            comment_lvl = find_elm("span",comment).text
            
            # sometimes 'level' not in span, catching that case below
            if len(comment_lvl.strip()) == 0:
                for text in comment.text.split("\n"):
                    li = text.find("level")
                    if li != -1:
                        comment_lvl = text
                        break
                        
            comment_lvl = int(comment_lvl.lower().replace("level","").strip())

        #if "1" in comment_lvl:
        if comment_lvl == 1:
            thread_count += 1
            user = comment.text.split("\n")[1]
            vote_count_txt = vote_div.text.replace(".","_")
            thread_id = f"{thread_count:04}-{user}-{vote_count_txt}"

            comment_id=0
        else:
            comment_id+=1

        # Save parent comments even if threshold is not met
        if comment_lvl > prev_comment_lvl:
            # entering a nested thread, mark new parent...
            parent_comment_stack.append(dict(comment=prev_comment,comment_id=prev_comment_id,saved=False))
        elif comment_lvl < prev_comment_lvl:
            # leaving a nested thread, remove previous parent
            parent_comment_stack = parent_comment_stack[:-1]


        # Check whether comment should be saved
        save_comment = False
        if vote_count >= vote_count_threshold:
            save_comment = True

        #save parent comment
        parent_comment = parent_comment_stack[-1]
        if save_comment and not parent_comment["saved"]:
            #todo refactor to fn
            folder = f"{asset_dir}/{submission_title}/{thread_id}"
            if not os.path.exists(folder):
                os.makedirs(folder)
            save_screenshot(parent_comment['comment'],f"{folder}/{parent_comment['comment_id']:04}.png")
            save_comment_text(filename=f"{folder}/{parent_comment['comment_id']:04}.txt",comment=parent_comment['comment'])
            parent_comment["saved"] = True

        #Save comment
        if save_comment:
            #thread_ids.append(thread_id)
            folder = f"{asset_dir}/{submission_title}/{thread_id}"
            if not os.path.exists(folder):
                os.makedirs(folder)
            save_screenshot(comment,f"{folder}/{comment_id:04}.png")
            if i == 0:
                save_screenshot(comment,f"{folder}/{comment_id:04}.png",scrollY=-55)
                save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment,selector='h1')
            else:
                save_screenshot(comment,f"{folder}/{comment_id:04}.png")
                save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment)

        # init vars for next iteration
        prev_comment_lvl = comment_lvl
        prev_comment = comment
        prev_comment_id = comment_id
        i+=1

getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


100%|█████████████████████████████████████████| 366/366 [03:43<00:00,  1.64it/s]


getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


100%|█████████████████████████████████████████| 501/501 [07:17<00:00,  1.15it/s]


getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


...

100%|█████████████████████████████████████████| 498/498 [03:57<00:00,  2.09it/s]


getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


100%|█████████████████████████████████████████| 406/406 [01:59<00:00,  3.39it/s]


getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


100%|█████████████████████████████████████████| 450/450 [01:37<00:00,  4.61it/s]


In [ ]:
#save_screenshot(commeants[1],f"{asset_dir}/test.png")

# Execute TTS at this point (on windows GPU) then run below
input("Execute TTS at this point (on windows GPU) then run below...(press enter)")

In [4]:
asset_dir = "/mnt/n/data/reddit-yt-tts/queued"
completed_dir="/mnt/n/data/reddit-yt-tts/completed"

In [5]:
submission_titles = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/*")]
submission_titles

['has_anyone_ever_been_to_a_wedding_where_someone',
 'what_was_normal_2030_years_ago_but_is_considered',
 'people_who_get_in_their_cars_but_then_take_ages',
 'to_you_redditors_aged_50_whats_something_you',
 'whats_something_popular_that_you_refuse_to_get',
 'what_is_just_not_the_flex_that_people_think_it_is',
 'people_who_wake_up_at_5am_daily_why',
 'what_television_series_had_the_biggest_bullshit']

In [3]:
#2023-05-22
post_urls = [
    "https://www.reddit.com/r/AskReddit/comments/13ec6op/has_anyone_ever_been_to_a_wedding_where_someone/",
    "https://www.reddit.com/r/AskReddit/comments/13i9ark/what_television_series_had_the_biggest_bullshit/",
    "https://www.reddit.com/r/AskReddit/comments/13lvmly/what_was_normal_2030_years_ago_but_is_considered/",
    "https://www.reddit.com/r/AskReddit/comments/13kuo93/to_you_redditors_aged_50_whats_something_you/",
    "https://www.reddit.com/r/AskReddit/comments/13angrv/whats_something_popular_that_you_refuse_to_get/",
    "https://www.reddit.com/r/AskReddit/comments/13nps5d/people_who_wake_up_at_5am_daily_why/",
    "https://www.reddit.com/r/AskReddit/comments/13bt91m/what_is_just_not_the_flex_that_people_think_it_is/",
    "https://www.reddit.com/r/AskReddit/comments/13hyxco/people_who_get_in_their_cars_but_then_take_ages/"
]


In [6]:
completed_metadata_files = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/../assets/*metadata*/*")]

In [7]:
skipped_threads = ["bernie_sanders_says_us_should_confiscate_100_of"]

In [8]:
for submission_title in submission_titles:
    
    #clean submission_title
    submission_title = submission_title.replace("â\x80\x99","'")
    print(f"Processing {submission_title} ...")
    post_url = list(filter(lambda x: submission_title in x,post_urls))[0] #index errors due to missing post_url 
    
    #skip completed files
    completed_metadata_files = [x.split("/")[-1].replace(".json","") for x in glob.glob(f"{asset_dir}/../assets/*metadata*/*")]
    if submission_title in completed_metadata_files:
        continue
        
    if submission_title in skipped_threads:
        continue
    thread_ids = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/{submission_title}/*")]

    def build_thread_vid(thread_id,bg_img="assets/Qv79akqGQt0.png"):
        images = glob.glob(f"{asset_dir}/{submission_title}/{thread_id}/*.png")
        images = sorted(images)
        audio_file_dir = f"{completed_dir}/{submission_title}/{thread_id}"
        # https://zulko.github.io/moviepy/examples/examples.html
        clips = []

        audio_clips = []
        total_duration = 0
        transition_duration = 0

        for i,image in enumerate(images):

            image_file = image.split("/")[-1]
            audio_file = image_file.replace(".png",".wav")
            try:
                audio_clip = mpy.AudioFileClip(f"{audio_file_dir}/{audio_file}").set_start(total_duration)
            except OSError as e:
                print(f"audio file {audio_file_dir}/{audio_file} not found")
                continue
            audio_clips.append(audio_clip)
            audio_duration = audio_clip.duration

            clip = mpy.ImageClip(image).set_duration(audio_duration).set_start(total_duration).set_position(("center", "center"))

            bg_clip = mpy.ImageClip(bg_img).set_duration(audio_duration).set_start(total_duration)
            if clip.size[0] > bg_clip.size[0] * .9:
                clip = clip.resize((bg_clip.size[0]*.9,clip.size[1]*.9))

            if clip.size[1] > bg_clip.size[1] * .9:
                clip = clip.resize((clip.size[0]*.9,bg_clip.size[1]*.9))
            #clip = clip.set_start(5) # TODO: change duration based on audio
            #clip = clip.resize((1080,1920)) #TODO add tts audio
            clips.append(bg_clip)
            clips.append(clip)

            total_duration += audio_duration + transition_duration

        if len(clips) > 0 :
            final_audio_clip = mpy.CompositeAudioClip(audio_clips)
            final_clip = mpy.CompositeVideoClip(clips)
            final_clip.audio = final_audio_clip
            final_clip.write_videofile(f"{asset_dir}/{submission_title}/{thread_id}/thread_clip.mp4",fps=30,logger=None)


    #import moviepy.video.fx as vfx
    def build_intro_vid(bg_img):
        #intro
        intro_duration = 4   
        bg_clip = mpy.ImageClip(bg_img).set_duration(intro_duration)
        bg_clip = mpy.vfx.fadein(bg_clip,intro_duration-1,[0,0,0])
        bg_clip.write_videofile(f"{asset_dir}/{submission_title}/intro.mp4",fps=30,logger=None)

    def build_outro_vid(bg_img):
        #outro
        outro_duration = 4   
        bg_clip = mpy.ImageClip(bg_img).set_duration(outro_duration)
        bg_clip = mpy.vfx.fadeout(bg_clip,outro_duration-1,[0,0,0])
        bg_clip.write_videofile(f"{asset_dir}/{submission_title}/outro.mp4",fps=30,logger=None)

    thread_ids = sorted(list(set(thread_ids)))

    f"{asset_dir}/{submission_title}/{thread_ids[0]}"

    bg_img="assets/bg_images/Qv79akqGQt0.png"

    # save threads to clips
    # vote_div.text

    thread_ids = sorted(list(set(thread_ids)))
    for thread in tqdm(thread_ids):
        print(thread)
        build_thread_vid(thread,bg_img)


    build_intro_vid(bg_img)
    build_outro_vid(bg_img)

    # save c to 

    videos = glob.glob(f"{asset_dir}/{submission_title}/*/*.mp4")
    videos = sorted(list(set(videos)))

    #prepend intro
    intro = glob.glob(f"{asset_dir}/{submission_title}/intro.mp4")
    videos = intro + videos

    #append outro
    outro = glob.glob(f"{asset_dir}/{submission_title}/outro.mp4")
    videos = videos + outro



    #videos

    all_video_clips = [mpy.VideoFileClip(video) for video in videos]

    videos_w_sound_corrected = []
    for clip in all_video_clips:
        if clip.audio:
            clip.audio = clip.audio.fx(mpy.afx.audio_normalize)
        videos_w_sound_corrected.append(clip)


    transition = mpy.VideoFileClip("assets/tvstatic-180-720.mp4")

    transition2 = mpy.VideoFileClip("assets/free-vhs-overlay-1080-720.mp4")

    transition2.audio = transition.audio
    transition2 = transition2.set_duration(.7)
    #transition = transition.resize(1080,720) resizing crashes kernal reesizing 
    #using ffmpeg on cli instead:  https://trac.ffmpeg.org/wiki/Scaling

    #split bg music into parts: https://unix.stackexchange.com/questions/280767/how-do-i-split-an-audio-file-into-multiple
    #ffmpeg -i somefile.mp3 -f segment -segment_time 3 -c copy out%03d.mp3


    final_video = mpy.concatenate_videoclips(videos_w_sound_corrected,method="compose",transition=transition2)

    def get_audio(v_duration):
        import random
        get_audio = True
        sel_file_clips = []
        sel_file_names = []
        while get_audio:

            audio_files = glob.glob("assets/bg_music/*.opus")

            sel_file = random.choice(audio_files)

            sel_file_clips.append(mpy.AudioFileClip(sel_file))
            sel_file_names.append(sel_file)

            all_duration = [ x.duration for x in sel_file_clips]

            total_duration = 0

            for duration in all_duration:
                total_duration += duration

            if total_duration >= v_duration:
                get_audio = False

        out_audio = mpy.concatenate_audioclips(sel_file_clips).set_duration(v_duration)
        return out_audio, sel_file_names 


    #import moviepy.video.fx as vfx
    #import moviepy.audio.fx as afx
    bg_music,bg_music_files = get_audio(final_video.duration)

    bg_music = bg_music.fx( mpy.afx.audio_normalize).fx( mpy.afx.volumex, 0.3).fx( mpy.afx.audio_fadein, 1.0)
    tts_audio = final_video.audio

    bgm_w_tts_audio = mpy.CompositeAudioClip([bg_music,tts_audio])

    final_video.audio = bgm_w_tts_audio

    final_video.write_videofile(f"assets/final_videos/{submission_title}.mp4",fps=30,logger=None)

    # Save video credits
    def check_wildcard(file,credit):
        prefix = credit["filename"].split(".*")[0]
        return prefix in file

    def get_audio_credits(audio_files):
        import yaml
        with open("assets/bg_music/asset_credits.yml","r") as f:
            credits = yaml.load(f,yaml.FullLoader)
        active_credits = []
        for credit in credits["assets"]:
            for file in audio_files:
                if ".*" in credit["filename"] and check_wildcard(file,credit):
                    active_credits.append(credit)
                elif credit["filename"] == file:
                    active_credits.append(credit)
        return active_credits

    credits = get_audio_credits(bg_music_files)
    all_credit_texts = []
    for credit in credits:
        credit_text = f"""
        - {credit['artist']} - {credit['title']} 
        - Provided by Lofi Girl
        - Watch: {credit['link']}
        - Listen: {credit['spotify']}
        """
        all_credit_texts.append(credit_text)
    audio_credits = "--------\n".join(set(all_credit_texts))
    #print(audio_credits)

    with open(f"assets/final_video_credits/{submission_title}.txt","w") as f:
        f.write(audio_credits)

    glob.glob(f"{asset_dir}/../archive_text/{submission_title}/*")

    from make_thumbnail import make_thumbnail_img
    prompt_file = glob.glob(f"{asset_dir}/../archive_text/{submission_title}/0001*/0000.txt")[0]

    with open(prompt_file,"r",encoding="latin1") as f:
        prompt_text = f.read()
    #prompt_text = "What did the pandemic ruin more than we realise?"    
    make_thumbnail_img(prompt_text,f"assets/final_video_thumbnails/{submission_title}.png",font="assets/fonts/limejuice/Limejuice.ttf")

    prompt_text

    import json

    metadata = dict(prompt_text=prompt_text
                    ,submission_title=submission_title
                    ,video_file=f"assets/final_videos/{submission_title}.mp4"
                    ,credits_file=f"assets/final_video_credits/{submission_title}.txt"
                   ,thumbnail=f"assets/final_video_thumbnails/{submission_title}.png"
                   ,post_url=post_url)

    with open(f"assets/final_video_metadata/{submission_title}.json","w") as f:
        json.dump(metadata,f)


    print(f"done!...info:\n prompt_text: {prompt_text}\n submission_title: {submission_title} \n video_file: assets/final_videos/{submission_title}.mp4 \n credits_file: assets/final_video_credits/{submission_title}.txt\n thumbnail: assets/final_video_thumbnails/{submission_title}.png \n post: {post_url}")

Processing has_anyone_ever_been_to_a_wedding_where_someone ...


  0%|                                                    | 0/56 [00:00<?, ?it/s]

0001-Posted by-30_5k


  2%|▊                                           | 1/56 [00:04<03:50,  4.19s/it]

0002-mynameizgary-10_4k


...

audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0064-Septixcake-430/0001.wav not found
0067-ZOINKSSSscoob-1_8k
audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0067-ZOINKSSSscoob-1_8k/0000.wav not found
0076-SnowdropWorks-566


 98%|██████████████████████████████████████████▏| 49/50 [11:54<00:00,  2.88it/s]

audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0076-SnowdropWorks-566/0000.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0076-SnowdropWorks-566/0001.wav not found
0078-gimp1615-475
audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0078-gimp1615-475/0000.wav not found


100%|███████████████████████████████████████████| 50/50 [11:54<00:00, 14.30s/it]


audio file /mnt/n/data/reddit-yt-tts/completed/what_television_series_had_the_biggest_bullshit/0078-gimp1615-475/0005.wav not found
done!...info:
 prompt_text: What television series had the biggest bullshit finale?
 submission_title: what_television_series_had_the_biggest_bullshit 
 video_file: assets/final_videos/what_television_series_had_the_biggest_bullshit.mp4 
 credits_file: assets/final_video_credits/what_television_series_had_the_biggest_bullshit.txt
 thumbnail: assets/final_video_thumbnails/what_television_series_had_the_biggest_bullshit.png 
 post: https://www.reddit.com/r/AskReddit/comments/13i9ark/what_television_series_had_the_biggest_bullshit/


In [20]:
glob.glob(f"{asset_dir}/../archive_text/{submission_title}/0001*/*")

['/mnt/n/data/reddit-yt-tts/queued/../archive_text/how_do_you_tell_your_gf_she_has_a_mustache/0001-Posted byu/[deleted]25 days ago-21_2k',
 '/mnt/n/data/reddit-yt-tts/queued/../archive_text/how_do_you_tell_your_gf_she_has_a_mustache/0001-Posted byu/0000.txt',
 '/mnt/n/data/reddit-yt-tts/queued/../archive_text/how_do_you_tell_your_gf_she_has_a_mustache/0001-Posted byu/0000.png']